In [4]:
import pickle
from statistics import mean
import numpy as np
import sys

In [5]:
file_name_train_250 = "../Human Experiments/Data/RL_trajectories_1000_250.pickle"
file_obj_train_250 = open(file_name_train_250, 'rb')
trajectories_train_250 = pickle.load(file_obj_train_250)
trajectories_train_250 = [list(ele) for ele in trajectories_train_250]

file_name_test_250 = "../Human Experiments/Data/Human_trajectories_test_250.pickle"
file_obj_test_250 = open(file_name_test_250, 'rb')
trajectories_test_250 = pickle.load(file_obj_test_250)
trajectories_test_250 = [list(ele) for ele in trajectories_test_250]

file_name_train_500 = "../Human Experiments/Data/RL_trajectories_1000_500.pickle"
file_obj_train_500 = open(file_name_train_500, 'rb')
trajectories_train_500 = pickle.load(file_obj_train_500)
trajectories_train_500 = [list(ele) for ele in trajectories_train_500]

file_name_test_500 = "../Human Experiments/Data/Human_trajectories_test_500.pickle"
file_obj_test_500 = open(file_name_test_500, 'rb')
trajectories_test_500 = pickle.load(file_obj_test_500)
trajectories_test_500 = [list(ele) for ele in trajectories_test_500]

test_size = 60

In [6]:
def generate_policies(price_low_bound, price_upper_bound):
    policies = []
    for i in range(price_low_bound, price_upper_bound, 10):
        for j in range(price_low_bound, price_upper_bound, 10):
            for k in range(price_low_bound, price_upper_bound, 10):
                for l in range(5):
                    for m in range(5):
                        for n in range(5):
                            policy = (i, j, k, l, m, n)
                            policies.append(policy)
    return policies

In [7]:
policies = generate_policies(150, 260)

In [8]:
len(policies)

166375

In [9]:
def evaluate_policy(policy, data, best_buys_cost, best_buys_idx, get_final_output):
    if(len(policy)!=6):
        raise ValueError("Number of parameters in this policy is "+ str(len(policy))+" when it should be 6")
    bought = []
    bought_cost_only = []
    bought_idx_only = []
    t1 = policy[0]
    t2 = policy[1]
    t3 = policy[2]
    d1 = policy[3]
    d2 = policy[4]
    d3 = policy[5]
    for trajectory in data:
        for idx, cost in enumerate(trajectory):
            local_cat = int(idx/5)
            local_idx = int(idx%5)
            if(idx == len(trajectory) - 1):
                bought.append((cost, idx))
                bought_cost_only.append(cost)
                bought_idx_only.append(idx)
                break
            elif(local_cat == 0):
                if(cost > t1 and local_idx <= d1):
                    continue
                else: 
                    bought.append((cost, idx))
                    bought_cost_only.append(cost)
                    bought_idx_only.append(idx)
                    break
            elif(local_cat == 1):
                if(cost > t2 and local_idx <= d2):
                    continue
                else: 
                    bought.append((cost, idx))
                    bought_cost_only.append(cost)
                    bought_idx_only.append(idx)
                    break
            elif(local_cat == 2):
                if(cost > t3 and local_idx <= d3):
                    continue
                else: 
                    bought.append((cost, idx))
                    bought_cost_only.append(cost)
                    bought_idx_only.append(idx)
                    break
    error = [a_i - b_i for a_i, b_i in zip(bought_cost_only, best_buys_cost)]
    correct_stops = (np.equal(bought_idx_only, best_buys_idx)).astype(int)
    correct_stops = sum(correct_stops)
    avg_error = sum(error)/len(data)
    if(get_final_output):
        return bought, avg_error, correct_stops
    return avg_error, correct_stops

In [10]:
##Cost
best_buy_250_train = [min(ele) for ele in trajectories_train_250]
best_buy_250_test = [min(ele) for ele in trajectories_test_250]
best_buy_500_train = [min(ele) for ele in trajectories_train_500]
best_buy_500_test = [min(ele) for ele in trajectories_test_500]

##Indices
best_buy_250_train_idx = [ele.index(min(ele)) for ele in trajectories_train_250]
best_buy_250_test_idx = [ele.index(min(ele)) for ele in trajectories_test_250]
best_buy_500_train_idx = [ele.index(min(ele)) for ele in trajectories_train_500]
best_buy_500_test_idx = [ele.index(min(ele)) for ele in trajectories_test_500]

In [11]:
evaluate_policy((190, 210, 230, 4, 4, 3), trajectories_train_250, best_buy_250_train, best_buy_250_train_idx, False)

(13.737, 524)

In [12]:
def policy_search(price_low_bound, price_upper_bound, data, best_buys_cost, best_buys_idx):
    all_errors = []
    all_correct_stops = []
    all_policies = generate_policies(price_low_bound, price_upper_bound)
    for idx, policy in enumerate(all_policies):
        policy_avg_error, correct_stops = evaluate_policy(policy, data, best_buys_cost, best_buys_idx, False)
        sys.stdout.flush()
        sys.stdout.write("\rPolicies Evaluated: {}/{}".format(idx+1, len(all_policies)))
        all_errors.append(policy_avg_error)
        all_correct_stops.append(correct_stops)
    optimal_policy_by_avg_error = (min(all_errors), all_policies[all_errors.index(min(all_errors))])
    optimal_policy_by_opt_stop = (max(all_correct_stops), all_policies[all_correct_stops.index(max(all_correct_stops))])
    return optimal_policy_by_avg_error, optimal_policy_by_opt_stop



# Route 1 (250)

In [150]:
optimal_policy_250 = policy_search(180, 231, trajectories_train_250, best_buy_250_train, best_buy_250_train_idx)
optimal_policy_250

Policies Evaluated: 27000/27000

((11.59, (200, 210, 230, 4, 4, 3)), (582, (200, 210, 220, 4, 4, 3)))

In [16]:
final_result_avg_250, avg_error, correct_stops = evaluate_policy((200, 210, 230, 4, 4, 3), 
                                                                 trajectories_test_250, 
                                                                 best_buy_250_test, 
                                                                 best_buy_250_test_idx, 
                                                                 True)
avg_error, correct_stops

(14.716666666666667, 25)

In [17]:
final_result_opt_stop_250, avg_error, correct_stops = evaluate_policy((200, 210, 220, 4, 4, 3), 
                                                                 trajectories_test_250, 
                                                                 best_buy_250_test, 
                                                                 best_buy_250_test_idx, 
                                                                 True)
avg_error, correct_stops

(17.483333333333334, 24)

# Route 2 (500)

In [146]:
optimal_policy_500 = policy_search(370, 501, trajectories_train_500, best_buy_500_train, best_buy_500_train_idx)

Policies Evaluated: 343000/343000

In [148]:
optimal_policy_500

((20.699, (410, 420, 460, 4, 4, 4)), (621, (400, 420, 430, 4, 4, 3)))

In [20]:
final_result_avg_500, avg_error, correct_stops = evaluate_policy((410, 420, 460, 4, 4, 4), 
                                                                 trajectories_test_500, 
                                                                 best_buy_500_test, 
                                                                 best_buy_500_test_idx, 
                                                                 True)
avg_error, correct_stops

(19.333333333333332, 34)

In [22]:
final_result_opt_stop_500, avg_error, correct_stops = evaluate_policy((400, 420, 430, 4, 4, 3), 
                                                                 trajectories_test_500, 
                                                                 best_buy_500_test, 
                                                                 best_buy_500_test_idx, 
                                                                 True)
avg_error, correct_stops

(30.9, 31)

# Final Result

In [23]:
rl_three_split_avg = [final_result_avg_250 + final_result_avg_500]
rl_three_split_opt_stop = [final_result_opt_stop_250 + final_result_opt_stop_500]

In [24]:
def save_data(obj, filename):
    final_file_object = open(filename, 'wb')
    pickle.dump(obj, final_file_object)
    final_file_object.close()

In [25]:
save_data(rl_three_split_avg, "../Human Experiments/Tests/rl_three_split_avg")
save_data(rl_three_split_opt_stop, "../Human Experiments/Tests/rl_three_split_opt_stop")